In [1]:
pip install pandas requests tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: C:\Users\gianluca\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [4]:
import os
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# --- 1. CONFIGURAZIONE PERCORSI E PARAMETRI ---

# Percorso assoluto alla root del progetto (due livelli su rispetto al notebook)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

CSV_PATH = os.path.join(PROJECT_ROOT, "data", "raw", "product_dataset.csv")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "data", "images", "products")

MAX_WORKERS = 20

# Crea la cartella se non esiste
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Verifica che il CSV esista prima di procedere
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(
        f"CSV non trovato in: {CSV_PATH}\n"
        f"Directory corrente: {os.getcwd()}\n"
        f"Assicurati che 'product_dataset.csv' sia in '{os.path.join(PROJECT_ROOT, 'data', 'raw')}'"
    )

# --- 2. FUNZIONE DI DOWNLOAD SINGOLO ---
def download_image(row):
    asset_id = row['product_asset_id']
    url = row['product_image_url']
    
    save_path = os.path.join(OUTPUT_DIR, f"{asset_id}.jpg")
    
    if os.path.exists(save_path):
        return True
        
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    except Exception:
        return False

# --- 3. ESECUZIONE MULTITHREADING ---
def main():
    print(f"📂 Project root: {PROJECT_ROOT}")
    print(f"Caricamento dataset da {CSV_PATH}...")
    
    df = pd.read_csv(CSV_PATH)
    df = df.dropna(subset=['product_image_url'])
    print(f"Trovate {len(df)} immagini da scaricare.")
    
    success_count = 0
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(download_image, row): row for _, row in df.iterrows()}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Download in corso"):
            if future.result():
                success_count += 1
                
    print(f"\n🚀 Download completato! {success_count}/{len(df)} immagini salvate in: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()

📂 Project root: C:\Users\gianluca\Desktop\HackUDC_2026
Caricamento dataset da C:\Users\gianluca\Desktop\HackUDC_2026\data\raw\product_dataset.csv...
Trovate 27688 immagini da scaricare.


Download in corso: 100%|█████████████████████████████████████████████████████████| 27688/27688 [08:21<00:00, 55.25it/s]


🚀 Download completato! 9026/27688 immagini salvate in: C:\Users\gianluca\Desktop\HackUDC_2026\data\images\products
